In [86]:
import gzip
from struct import unpack, pack, calcsize
from datetime import timedelta
from collections import deque
import numpy as np

In [2]:
# Offset + length_last - 1
# These values are taken from 
message_type_size_dict = {
    b"S" : 11, b"R" : 38, b"H" : 24, b"Y" : 19, b"L" : 25, b"V" : 34, b"W" : 11, b"K" : 27, b"J" : 34, b"h" : 20, b"A" : 35,
    b"F" : 39, b"E" : 30, b"C" : 35, b"X" : 22, b"D" : 18, b"U" : 34, b"P" : 43, b"Q" : 39, b"B" : 18, b"I" : 49, b"N" : 19,
    b"\x00" : 1
}

In [3]:
market_open='0930'
market_close = '1600'

In [25]:
def get_timestamp(binary_value):
    time_stamp = unpack('>6s', binary_value)[0];
    return int.from_bytes(time_stamp, byteorder='big')
    
def handle_system_message():
    global market_o, market_s, market_q, market_m, market_e, market_c
    
    # System Message
    stock_locate, tracking_number = unpack('>HH', msg[0:4])
    event_code = unpack('>c', msg[10:])[0]
    time_stamp = get_timestamp(msg[4:10])
    if event_code == b"O":
        market_o = market_o + 1
        # Start of Messages.Outside oftime stamp messages,the start of day message isthe first message sentin
        # any trading day
    elif event_code == b"S":
        market_s = market_s + 1
        # Start of System hours. This message indicatesthatNASDAQis open and ready to start accepting orders
    elif event_code == b"Q":
        market_q = market_q + 1
        # Start of Market hours. This message isintended to indicate that Market Hours orders are available
        # for execution
    elif event_code == b"M":
        market_m = market_m + 1
        # End of Market hours. This message isintended to indicate that Market Hours orders are no longer
        # available for execution
    elif event_code == b"E":
        market_e = market_e + 1
        # End of System hours. It indicates that Nasdaq is now closed and will not accept any new orderstoday.
        # It is still possible to receive Broken Trade messages and Order Delete messages after the End of Day
    elif event_code == b"C":
        market_c = market_c + 1
        # End of Messages. This is always the last message sent in any trading day.
    converted_time = timedelta(seconds=time_stamp * 1e-9)
    print (stock_locate, tracking_number, converted_time, event_code)

In [130]:
def handle_Add_Order_message(bMPID = False):
    stock = unpack('>8s', msg[23:31])[0]
    time_stamp = get_timestamp(msg[4:10])
    shares = unpack('>I', msg[19:23])[0]
    price = unpack('>I', msg[31:35])[0] * (1e-4)
    converted_time = timedelta(seconds=time_stamp * 1e-9)
    order_ref_number = unpack('>Q', msg[10:18])[0]
    ref_number_stocks.append([order_ref_number, stock])
    

def get_stock(order_ref_number):
    for i in range(len(ref_number_stocks)):
        if ref_number_stocks[i][0] == order_ref_number:
            return (ref_number_stocks[i][1])

def handle_Order_Execute_message():
    printable = unpack('>c', msg[30:31])[0]
    if printable != b'N':
        time_stamp = get_timestamp(msg[4:10])
        shares = unpack('>I', msg[18:22])[0]

        price = unpack('>I', msg[31:])[0] * (1e-4)
        order_ref_number = unpack('>Q', msg[10:18])[0]
        converted_time = timedelta(seconds=time_stamp * 1e-9)
        stock = get_stock(order_ref_number)

        print (order_ref_number, stock, converted_time, shares, price, printable)

        if stock not in stocks_current_avg:
            stocks_past_cummulative_volume[stock] = 0
            stocks_past_cummulative_TPV[stock] = 0

        if stock not in stocks_times:
            stocks_times[stock] = deque()
            
        stock_queue = stocks_times[stock]
        stock_queue.append({'time_stamp' : time_stamp,
                            'price' : price})

        while stock_queue[0]['time_stamp'] < (time_stamp - tick_time): 
            stock_queue.popleft()
        
        N = len(stock_queue)
        high = price
        low = price
        for i in range(N):
            if stock_queue[i]['price'] > high:
                high = stock_queue[i]['price']
            if stock_queue[i]['price'] < low:
                low = stock_queue[i]['price']
          
        print (">>>>>>>>>>>>>>>> ", N, high, low, price)
        typical_price = (high + low + price)/3
        current_PV = shares * typical_price
        
        stocks_past_cummulative_TPV[stock] = (stocks_past_cummulative_TPV[stock] +  
                                              current_PV) / (stocks_past_cummulative_volume[stock] + shares)
        stocks_past_cummulative_volume[stock] = shares
        
        print (stock, stocks_past_cummulative_TPV[stock], converted_time)

In [131]:
bin_data = open("20190130.PSX_ITCH_50", "rb")
message_type = bin_data.read(1);
market_o, market_s, market_q, market_m, market_e, market_c = (0, 0, 0, 0, 0, 0)
number_of_messages = 0
ref_number_stocks = []
stocks_times = {}
stocks_current_avg = {}
stocks_past_cummulative_volume = {}
stocks_past_cummulative_TPV = {}
 # rolling average of Min = 6e10 --- Hour = 3.6e12
tick_time = 6e10
while message_type:
    msg = bin_data.read(message_type_size_dict[message_type])
    number_of_messages += 1
    # trading Message
    if message_type == b"A":
        handle_Add_Order_message()
    elif message_type == b"F":
        handle_Add_Order_message(bMPID = True)
    elif message_type == b"C":
        handle_Order_Execute_message()
    elif message_type == b"S":
        handle_system_message()
    message_type = bin_data.read(1);
    if number_of_messages > 1000000: # for all change to 100000000 or remove this if and break!
        break
    
print (number_of_messages, market_o, market_s, market_q, market_m, market_e, market_c)

0 0 3:06:20.838940 b'O'
0 0 8:00:00.000025 b'S'
0 0 9:30:00.000077 b'Q'
274625 b'EMB     ' 9:30:00.168245 10 107.65 b'Y'
>>>>>>>>>>>>>>>>  1 107.65 107.65 107.65
b'EMB     ' 107.65000000000002 9:30:00.168245
278741 b'XBI     ' 9:30:00.301322 100 80.60000000000001 b'Y'
>>>>>>>>>>>>>>>>  1 80.60000000000001 80.60000000000001 80.60000000000001
b'XBI     ' 80.60000000000001 9:30:00.301322
280091 b'BND     ' 9:30:00.353285 4 79.60000000000001 b'Y'
>>>>>>>>>>>>>>>>  1 79.60000000000001 79.60000000000001 79.60000000000001
b'BND     ' 79.60000000000001 9:30:00.353285
287675 b'OGE     ' 9:30:00.675519 600 40.03 b'Y'
>>>>>>>>>>>>>>>>  1 40.03 40.03 40.03
b'OGE     ' 40.03 9:30:00.675519
287675 b'OGE     ' 9:30:00.738494 100 40.03 b'Y'
>>>>>>>>>>>>>>>>  2 40.03 40.03 40.03
b'OGE     ' 40.03 9:30:00.738494
293554 b'IWM     ' 9:30:00.963590 30 146.96 b'Y'
>>>>>>>>>>>>>>>>  1 146.96 146.96 146.96
b'IWM     ' 146.96 9:30:00.963590
293554 b'IWM     ' 9:30:00.963596 70 146.96 b'Y'
>>>>>>>>>>>>>>>>  2 1

In [6]:
calcsize('>HHhL8ccc4c')
calcsize('>HHhLc')
calcsize('HHI')

# Making a 6 byte timestamp
def unpack48(x):
    x1, x2, x3 = struct.unpack('<HHI', x)
    return x1, x2 | (x3 << 16)
calcsize('HH6cc')

11

In [7]:
(1024).to_bytes(2, byteorder='big')

b'\x04\x00'

In [37]:
bin_data = open("20190130.PSX_ITCH_50", "rb")
message_type = bin_data.read(1)
number_of_messages = 0
while message_type:
    msg = bin_data.read(message_type_size_dict[message_type])
    if message_type == b'C':
        handle_Order_Execute_message()
    message_type = bin_data.read(1);
    
    if number_of_messages > 1000000: # for all change to 100000000 or remove this if and break!
        break
    number_of_messages += 1

(274625,) 9:30:00.168245 (10,) 107.65 b'Y'
(278741,) 9:30:00.301322 (100,) 80.60000000000001 b'Y'
(280091,) 9:30:00.353285 (4,) 79.60000000000001 b'Y'
(287675,) 9:30:00.675519 (600,) 40.03 b'Y'
(287675,) 9:30:00.738494 (100,) 40.03 b'Y'
(293554,) 9:30:00.963590 (30,) 146.96 b'Y'
(293554,) 9:30:00.963596 (70,) 146.96 b'Y'
(305869,) 9:30:01.933280 (71,) 101.66000000000001 b'Y'
(280091,) 9:30:05.004557 (49,) 79.60000000000001 b'Y'
(336106,) 9:30:09.676567 (100,) 15.56 b'Y'
(336106,) 9:30:09.684657 (18,) 15.56 b'Y'
(336372,) 9:30:09.741860 (100,) 92.30000000000001 b'Y'
(337011,) 9:30:09.892260 (85,) 43.25 b'Y'
(351246,) 9:30:13.794988 (100,) 111.56 b'Y'
(368162,) 9:30:20.250780 (100,) 63.38 b'Y'
(368164,) 9:30:20.250780 (100,) 63.38 b'Y'
(367067,) 9:30:22.033316 (100,) 53.760000000000005 b'Y'
(382852,) 9:30:26.265546 (100,) 15.790000000000001 b'Y'
(385795,) 9:30:27.340699 (100,) 70.4 b'Y'
(414231,) 9:30:36.885068 (100,) 39.11 b'Y'
(428641,) 9:30:42.980658 (100,) 38.52 b'Y'
(432039,) 9:30:4